# Python, Crontab 의 활용의 예 [data 수집]

# Dart 재무재표 수집


<center>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FwM9Ci%2Fbtq7sDdrXpS%2FbwNkV4jivSR15XVXjljIcK%2Fimg.png" width="900" height="900"><br>
</center>

#### 파이썬 안에서 또, 파이썬 파일을 만들어 이를 스크립트로 실행 하는 구조 

- 이렇게 했을때의 장점으로는, 메모리에 과부하가 걸리는 현상을 사전에 막을 수 있다.


## 1. main.py 

os.system()으로 커맨드에서 python을 실행 시킨다. 그 다음 for loop안에 각각의 python코드가 실행하는 방법이다.

In [ ]:
import os
import time
import pandas as pd

In [ ]:
date_list_start = ["20210101", "20210401"]
date_list_end = ["20210331", "20210615"]

apikey = [
    "google api key", # google 계정
    "naver api key" # naver 계정
]

# 1분기보고서 : 11013
# 반기보고서 : 11012
# 3분기보고서 : 11014
# 사업보고서 : 11011

report_type = ["11013", "11012", "11014", "11011"]

for j in range(2):
    for i in report_type:
        os.system("/usr/bin/python3 /home/ubuntu/python/Python_Example/dart_crawl.py" + " " + date_list_start[j] + " " + date_list_end[j] + " " +  i + " " + apikey[j])

위와 같이 코드를 구성할때 파이썬은 다음과 같은 커맨드 입력을 하여 python파일을 호출하게 된다. 여기서 dart_crawl.py 파일을 호출하는 코드인데 뒤에 date_list_start, date_list_end, apikey, report_type를  dart_crawl.py에 인자로 넘겨준다.

## 2. func.py

main.py에서 호출받아 실행하게 되는 dart_crawl.py는 main.py에서 변수를 받아 이곳에 적용하게 된다. 여기서는 date_list_start, date_list_end, apikey, report_type 변수를 main.py에서 받았으며, 이를 func.py에서 적용시켜주기 위해서 각각 sys.argv[1],[2],[3],[4]를 사용한다.

In [1]:
# OpenDartReader를 사용한 Dart 재무제표 수집
import datetime
import time
import pandas as pd
import numpy as np
import OpenDartReader
import os
import sys

In [ ]:
argv_value_1 = sys.argv[1] # date start
argv_value_2 = sys.argv[2] # date end
argv_value_3 = sys.argv[3] # report type
argv_value_4 = sys.argv[4] # api key

In [ ]:
# ==== 0. 객체 생성 ====
# 객체 생성 (API KEY 지정) 
api_key = argv_value_4

dart = OpenDartReader(api_key) 

In [ ]:
report_list = dart.list(start=argv_value_1, end=argv_value_2, kind='A')
corp_list = report_list['corp_name']

In [ ]:
# 파일명

file_name = "file_" + argv_value_1 + "_" + argv_value_2 + "_" + argv_value_3 + ".csv"
time_log = "log_" + argv_value_1 + "_" + argv_value_2 + "_" + argv_value_3 + ".csv"

dart_crawl.py안에 main.py에서 받은 변수값에 대한 결과 값을 로그로 남겨보려 한다. datetime모듈을 사용였으며 to_csv()의 mode=’a’옵션을 사용하여 다음과 같이 csv를 누적으로 저장해서 눈으로 볼 수 있게 만들었다.

In [ ]:
for i in corp_list:
    # 분당 100개 제한 관련 함수
    time.sleep(3)
    
    # log 에 찍을 시각
    now = datetime.datetime.now()
    
    # 21년도 재무재표 수집
    try:
        df1 = dart.finstate(i, 2021, reprt_code=argv_value_3)
    except Exception as e:
        df_log = pd.DataFrame({
            'TF': '회사명 에러',
            '회사이름':i,
            'time':[now]})
    
        df_log.to_csv(time_log, index=False, mode = "a", header=False)

    if df1 is None:
        # 데이터 존재 하지 않을 경우 로그    
        df_log = pd.DataFrame({
            'TF':"존재하지 않음",
            '회사이름':i,
            'time':[now]})
    
        df_log.to_csv(time_log, index=False, mode = "a", header=False)
    else:
        # 데이터가 존재할경우 csv에 저장
        if not os.path.exists(file_name):
            df1.to_csv(file_name, index=False, mode='w')
        else:
            df1.to_csv(file_name, index=False, mode='a', header=False)

            df_log = pd.DataFrame({
                'TF':"성공",
                '회사이름':i,
                'time':[now]})
            df_log.to_csv(time_log, index=False, mode = "a", header=False)

### vi 편집기 화면 (main.py, dart_crawl.py)


<center>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FtPHcz%2Fbtq7DUfC94b%2FLbMRkcgw6eKJ5rnpVd7BeK%2Fimg.png" width="900" height="900"><br>
</center>


<center>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FWIF7q%2Fbtq7KmnVVRa%2FGPS5nEqhofDqrQlKwbtuvk%2Fimg.png" width="900" height="900"><br>
</center>



## 결과

<center>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fdr2IXj%2Fbtq7Fg92PFo%2FQzo1ASjUWdGRl9NF09MVbK%2Fimg.png" width="900" height="900"><br>
</center>


<center>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fcrnluv%2Fbtq7EernMtz%2FhSwVVdiOKE0Vl9l9QYvol1%2Fimg.png" width="900" height="900"><br>
</center>

- opendartreader API를 사용했고, 하루에 10000개 이상 받아 올 수 없어서, # 분당 100개 제한 관련 함수 time.sleep(3)를 사용했다. 
- Crontab 의 자동화의 편리함과 왜 사용하는지 알 수 있었다. 좀 더 공부하고 응용을 해서 세련되고 편리하게 자동화를 구축해 보자.

#### Reference

- [명령어 인자 및 응용](https://unfinishedgod.netlify.app/2021/06/16/python-%EB%AA%85%EB%A0%B9%EC%96%B4-%EC%9D%B8%EC%9E%90-%EB%B0%8F-%EC%9D%91%EC%9A%A9/)